In [1]:
import cv2
import numpy as np
import skimage.feature as skf
import skimage.color as skc
from skimage.filters import sobel
from skimage.transform import resize
import json
import os
from sklearn.decomposition import PCA
from elasticsearch import Elasticsearch
import matplotlib.pyplot as plt
import pywt

In [2]:
# dataset_folder_path = "full_dataset"
dataset_folder_path = "dataset"

# Define image size dictionary
img_size = {'x': 128, 'y': 128}
bin_count = 16

ELASTIC_PASSWORD = "random_elastic_password"
ELASTIC_INDEX = 'image_processing_test_db'
# ELASTIC_INDEX = 'image_processing_db'

# Initialize Elasticsearch client
es = Elasticsearch('https://elastic.bagheriali.net',
    basic_auth=("elastic", ELASTIC_PASSWORD)
)
es.info()

ObjectApiResponse({'name': 'elastic', 'cluster_name': 'docker-cluster', 'cluster_uuid': '5ypOTvpkTnuzEnHTo80PeQ', 'version': {'number': '8.17.0', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '2b6a7fed44faa321997703718f07ee0420804b41', 'build_date': '2024-12-11T12:08:05.663969764Z', 'build_snapshot': False, 'lucene_version': '9.12.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [3]:
# Function to add feature to Elasticsearch
def index_feature_to_elasticsearch(feature_dict):
    file_name = feature_dict['file']
    es.options(request_timeout=30).index(index=ELASTIC_INDEX, id=file_name, body=feature_dict)

def set_default(obj):
    if isinstance(obj, set):  # Convert sets to lists
        return list(obj)
    if isinstance(obj, np.ndarray):  # Convert NumPy arrays to lists
        return obj.tolist()
    if isinstance(obj, np.generic):  # Handle NumPy scalar types (e.g., np.float32, np.int32)
        return obj.item()
    if isinstance(obj, cv2.KeyPoint):  # Convert KeyPoint to a dictionary
        return {
            "x": obj.pt[0],   # X coordinate
            "y": obj.pt[1],   # Y coordinate
            "size": obj.size, # Size of the keypoint
            "angle": obj.angle,  # Angle of orientation
            "response": obj.response,  # Strength of keypoint
            "octave": obj.octave,  # Pyramid layer
            "class_id": obj.class_id  # Object class ID
        }
    raise TypeError(f"Type {type(obj)} not serializable")


In [4]:
# Pre-processing Functions
def resize_image(image, size=(img_size['x'], img_size['y'])):
    return cv2.resize(image, size)

def convert_to_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

def denoise_image(image):
    return cv2.fastNlMeansDenoisingColored(image, None, 10, 10, 7, 21)

def edge_detection(image):
    gray = convert_to_grayscale(image)
    return cv2.Canny(gray, 100, 200)

############## FULLY COMPLETED ##############

# 1. RGB Color Mean
def color_intensity_mean(image):
    b_mean, g_mean, r_mean = cv2.mean(image)[:3]  # Extract RGB mean values
    gray = convert_to_grayscale(image)
    intensity_mean = np.mean(gray)
    return r_mean, g_mean, b_mean, intensity_mean

# 2. RGB Color Histogram
def color_intensity_histogram(image, bins=bin_count):
    # Compute histograms for each channel separately
    blue_hist = cv2.calcHist([image], [0], None, [bins], [0, 256]).flatten()
    green_hist = cv2.calcHist([image], [1], None, [bins], [0, 256]).flatten()
    red_hist = cv2.calcHist([image], [2], None, [bins], [0, 256]).flatten()

    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    intensity_hist = cv2.calcHist([gray], [0], None, [bins], [0, 256]).flatten()
    
    # Normalize histograms
    blue_hist /= blue_hist.sum()
    green_hist /= green_hist.sum()
    red_hist /= red_hist.sum()
    intensity_hist /= intensity_hist.sum()
    
    return red_hist, green_hist, blue_hist, intensity_hist

# 3. GLCM Features
def glcm_features(image, dx, dy):
    gray = convert_to_grayscale(image)
    gray = (gray * 255).astype(np.uint8)  # Convert to uint8 for GLCM compatibility
    glcm = skf.graycomatrix(gray, distances=[1], angles=[np.arctan2(dy, dx)], levels=256, symmetric=True, normed=True)
    features = {
        'energy': skf.graycoprops(glcm, 'energy')[0, 0],
        'contrast': skf.graycoprops(glcm, 'contrast')[0, 0],
        'entropy': -np.sum(glcm * np.log2(glcm + (glcm == 0))),
        'dissimilarity': skf.graycoprops(glcm, 'dissimilarity')[0, 0],
        'homogeneity': skf.graycoprops(glcm, 'homogeneity')[0, 0],
        'correlation': skf.graycoprops(glcm, 'correlation')[0, 0]
    }
    return features

def glcm_features_all_directions(image):
    directions = [(1, 0), (0, 1), (1, 1), (-1, 1)]  # Right, Down, Diagonal, Anti-diagonal

    energy = []
    contrast = []
    entropy = []
    dissimilarity = []
    homogeneity = []
    correlation = []

    for dx, dy in directions:
        features = glcm_features(image, dx, dy)
        energy.append(features["energy"])
        contrast.append(features["contrast"])
        entropy.append(features["entropy"])
        dissimilarity.append(features["dissimilarity"])
        homogeneity.append(features["homogeneity"])
        correlation.append(features["correlation"])

    return {
        "energy": np.array(energy),
        "contrast": np.array(contrast),
        "entropy": np.array(entropy),
        "dissimilarity": np.array(dissimilarity),
        "homogeneity": np.array(homogeneity),
        "correlation": np.array(correlation)
    }

# 4. HOG
def compute_hog(image, pixel_per_cell=16, cell_per_block=2, orientations=6):
    gray = convert_to_grayscale(image)
    hog = skf.hog(gray, pixels_per_cell=(pixel_per_cell, pixel_per_cell), 
        cells_per_block=(cell_per_block, cell_per_block), 
        orientations=orientations, feature_vector=True)
    # return hog
    
    hog_rounded = np.vectorize(lambda x: float(f"{x:.6f}"))(hog)
    return hog_rounded

# 5. GIST Descriptor (Simplified using Sobel)
def compute_gist(image, size=(img_size['x']//4, img_size['y']//4)):
    resized = resize(image, size)
    gray = skc.rgb2gray(resized)
    edges = sobel(gray)
    return edges.flatten()

# 6. DCT Features
def extract_dct_features(image, block_size=16, num_coefficients=20):
    # Convert the image to grayscale if it's a color image
    if len(image.shape) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Normalize the image to range [0, 255] if it's not already
    image = np.float32(image) / 255.0
    
    # Prepare an empty list to store DCT coefficients
    dct_features = []

    # Iterate through the image in blocks
    height, width = image.shape
    for i in range(0, height, block_size):
        for j in range(0, width, block_size):
            # Extract a block of the image
            block = image[i:i+block_size, j:j+block_size]
            
            # Apply DCT on the block (2D DCT)
            dct_block = cv2.dct(block)
            
            # Flatten the block's DCT coefficients and add to feature list
            dct_features.extend(dct_block.flatten()[:num_coefficients])

    # Return the first 'num_coefficients' from the whole image's DCT blocks as features
    return np.array(dct_features)

# 7. Wavelet Features
def extract_wavelet_features(image, wavelet='db1', level=3):
    # Perform Discrete Wavelet Transform (DWT)
    coeffs2 = pywt.dwt2(image, wavelet, level)
    
    # Extract sub-bands from the wavelet decomposition
    cA, (cH, cV, cD) = coeffs2
    
    # You can perform further decomposition on each of the sub-bands if needed
    # For simplicity, we'll focus on the first level decomposition.
    
    # Flatten the coefficients into one-dimensional arrays
    features = []
    
    # Calculate features like energy, standard deviation, etc. from each sub-band
    for coeff in [cA, cH, cV, cD]:
        energy = np.sum(coeff**2)  # Energy of the sub-band
        std_dev = np.std(coeff)    # Standard deviation of the sub-band
        mean = np.mean(coeff)      # Mean of the sub-band
        
        # Append these features to the list
        features.extend([energy, std_dev, mean])
    
    # Convert the feature list to a numpy array
    return np.array(features)

# 8. Harris Corner Detector
def harris_corners(image, block_size=2, ksize=3, k=0.04, threshold=0.1):
    image2 = image.copy()
    image2 = resize_image(image2, (32, 32))

    # Convert the image2 to grayscale
    gray = convert_to_grayscale(image2)
    gray = np.float32(gray)
    
    # Apply the Harris corner detection
    corners = cv2.cornerHarris(gray, block_size, ksize, k)
    # Apply threshold to corners
    corners = (corners > threshold * corners.max()).astype(np.uint8) * 255
    
    # return corners
    # Convert the 2D corner array to a 1D array
    corners_1d = corners.flatten()  # or np.ravel()
    
    return corners_1d

# TODO: ADD SURF

############ NOT FULLY COMPLETED ############


# 9. SIFT
def compute_sift(image):
    gray = convert_to_grayscale(image)
    gray = (gray * 255).astype(np.uint8)  # Convert back to uint8
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(gray, None)
    # print("descriptors ", descriptors)
    return descriptors

    # # pca = PCA(n_components=0.95)  # Keep top 50 principal components
    # pca = PCA(n_components=8)  # Keep top 50 principal components
    # compressed_descriptors = pca.fit_transform(descriptors)
    # return compressed_descriptors

def get_features (image, image_path):
    image = resize_image(image)
    image = denoise_image(image)

    convert_to_6_digit_float = np.vectorize(lambda x: float(f'{x:.6f}'))

    r_mean, g_mean, b_mean, i_mean = color_intensity_mean(image)
    r_hist, g_hist, b_hist, i_hist = color_intensity_histogram(image)
    features = glcm_features_all_directions(image)
    hog = compute_hog(image)
    gist = compute_gist(image)
    dct = extract_dct_features(image)
    wavelet = extract_wavelet_features(image)
    corners = harris_corners(image)
    # sift = compute_sift(image)


    # print("dct: ", type(dct), dct.shape)
    # print("wavelet: ", type(wavelet), wavelet.shape)
    # print("corners: ", type(corners), corners.shape)

    feature = {
        'file': image_path,
        'r_mean': float(f'{r_mean:.6f}'),
        'g_mean': float(f'{g_mean:.6f}'),
        'b_mean': float(f'{b_mean:.6f}'),
        'i_mean': float(f'{i_mean:.6f}'),
        'r_hist': convert_to_6_digit_float(r_hist),
        'g_hist': convert_to_6_digit_float(g_hist),
        'b_hist': convert_to_6_digit_float(b_hist),
        'i_hist': convert_to_6_digit_float(i_hist),
        'energy': convert_to_6_digit_float(features['energy']),
        'contrast': convert_to_6_digit_float(features['contrast']),
        'entropy': convert_to_6_digit_float(features['entropy']),
        'dissimilarity': convert_to_6_digit_float(features['dissimilarity']),
        'homogeneity': convert_to_6_digit_float(features['homogeneity']),
        'correlation': convert_to_6_digit_float(features['correlation']),
        'hog': convert_to_6_digit_float(hog),
        'gist': convert_to_6_digit_float(gist),
        'dct': dct,
        'wavelet': wavelet,
        'corners': corners,
        # 'sift': [{"vector": vec.tolist()} for vec in sift],
        # 'sift_descriptors': sift.tolist(),
    }

    # features_list.append({
    #     'opponent_mean': opponent_color_mean(image),
    #     'opponent_hist': opponent_color_histogram(image),
    #     'harris_pca': harris_corners(image),
    #     'sift_pca': compute_sift(image)
    # })

    return feature


In [ ]:
# image_path = 'dataset/benches_and_chairs-153_5361.JPG'
image_path = 'dataset/aeroplanes-general-166_6630.JPG'
image = cv2.imread(image_path)
_ = get_features(image, image_path)

In [ ]:
# Process images in a folder
def process_images_in_folder(folder_path):
    features_list = []
    i = 1
    for filename in os.listdir(folder_path):
        print(f"{i} - File to Process: {filename}   -   -   -   ", end="\r")
        if filename.lower().endswith(('png', 'jpg', 'jpeg', 'bmp')):
            image_path = os.path.join(folder_path, filename)
            image = cv2.imread(image_path)
            if image is None:
                continue
            
            feature = get_features(image, filename)
            
            features_list.append(feature)
        if i == 10:
            break
        i += 1
    return features_list

# Modify process_images_in_folder to include indexing
def process_images_in_folder_to_elastic(folder_path):
    i = 1
    process_thangs = False
    files = os.listdir(folder_path)
    files.sort()

    for filename in files:
        # print(f"{i} {process_thangs} - File to Process: {filename}   -   -   -   ")
        print(f"{i} {process_thangs} - File to Process: {filename}   -   -   -   ", end="\r")
        if (process_thangs):
            # if filename.lower().endswith(('png', 'jpg', 'jpeg', 'bmp')) and not (filename in ["cars-side-view-104_0428.JPG", "windows-180_8006.JPG"]):
            if filename.lower().endswith(('png', 'jpg', 'jpeg', 'bmp')) and not (filename in ["buildings-118_1832.JPG", "cars-side-view-179_7912.JPG"]):
                image_path = os.path.join(folder_path, filename)
                image = cv2.imread(image_path)
                if image is None:
                    continue
                
                feature = get_features(image, filename)

                index_feature_to_elasticsearch(feature)  # Index in Elasticsearch
        elif i == 1060:
            process_thangs = True
            
        i += 1

results = process_images_in_folder(dataset_folder_path)
print(f"Writing the data to results.json")
with open('results.json', 'w', encoding='utf-8') as f:
    json.dump(results, f, default=set_default, ensure_ascii=False, indent=4)
process_images_in_folder_to_elastic(dataset_folder_path)


In [ ]:

# Function to search for similar images
def search_similar_images(image_path, feature_keys=['hist'], top_n=10):
    image = cv2.imread(image_path)
    image_rgb = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image_rgb, cv2.COLOR_BGR2RGB)
    if image is None:
        print("Error: Unable to read image.")
        return []
    
    image = resize_image(image)
    image = denoise_image(image)

    query_features = {}

    if 'mean' in feature_keys:
        r_mean, g_mean, b_mean, i_mean = color_intensity_mean(image)
        query_features['r_mean'] = r_mean
        query_features['g_mean'] = g_mean
        query_features['b_mean'] = b_mean
        query_features['i_mean'] = i_mean

    if 'hist' in feature_keys:
        r_hist, g_hist, b_hist, i_hist = color_intensity_histogram(image)
        query_features['r_hist'] = r_hist.tolist()
        query_features['g_hist'] = g_hist.tolist()
        query_features['b_hist'] = b_hist.tolist()
        query_features['i_hist'] = i_hist.tolist()

    if 'glcm' in feature_keys :
        features = glcm_features_all_directions(image)
        query_features['energy'] = features['energy'].tolist()
        query_features['contrast'] = features['contrast'].tolist()
        query_features['entropy'] = features['entropy'].tolist()
        query_features['dissimilarity'] = features['dissimilarity'].tolist()
        query_features['homogeneity'] = features['homogeneity'].tolist()
        query_features['correlation'] = features['correlation'].tolist()

    if 'hog' in feature_keys:
        hog = compute_hog(image)
        query_features['hog'] = hog

    if 'gist' in feature_keys:
        gist = compute_gist(image)
        query_features['gist'] = gist

    if 'dct' in feature_keys:
        dct = extract_dct_features(image)
        query_features['dct'] = dct

    if 'wavelet' in feature_keys:
        wavelet = extract_wavelet_features(image)
        query_features['wavelet'] = wavelet

    if 'corners' in feature_keys:
        corners = harris_corners(image)
        query_features['corners'] = corners


    query = {
        "size": top_n,
        "query": {
            "script_score": {
                "query": { "match_all": {} },
                "script": {
                    "source": """
                        double total_cosineSimilarity = 0.0;
                        double total_weight = 0.0;
                        
                        if (params.containsKey('r_mean')) {
                            // Query values
                            double a1 = params.r_mean;
                            double a2 = params.g_mean;
                            double a3 = params.b_mean;
                            double a4 = params.i_mean;

                            // Document values
                            double b1 = params._source.r_mean;
                            double b2 = params._source.g_mean;
                            double b3 = params._source.b_mean;
                            double b4 = params._source.i_mean;

                            // Compute dot product
                            double dot_product = (a1 * b1) + (a2 * b2) + (a3 * b3) + (a4 * b4);

                            // Compute norms
                            double query_norm = Math.sqrt((a1 * a1) + (a2 * a2) + (a3 * a3) + (a4 * a4));
                            double doc_norm = Math.sqrt((b1 * b1) + (b2 * b2) + (b3 * b3) + (b4 * b4));

                            // Compute cosine similarity
                            total_cosineSimilarity += (query_norm * doc_norm == 0) ? 0 : dot_product / (query_norm * doc_norm);
                            total_weight += (query_norm * doc_norm == 0) ? 0 : 1;
                        }
                        if (params.containsKey('r_hist')) {
                            total_cosineSimilarity += (cosineSimilarity(params.r_hist, 'r_hist') + cosineSimilarity(params.g_hist, 'g_hist') + cosineSimilarity(params.b_hist, 'b_hist') + cosineSimilarity(params.i_hist, 'i_hist')) / 4 * 1;
                            total_weight += 1;
                        }
                        if (params.containsKey('energy')) {
                            total_cosineSimilarity += (cosineSimilarity(params.energy, 'energy') + cosineSimilarity(params.contrast, 'contrast') + cosineSimilarity(params.entropy, 'entropy') + cosineSimilarity(params.dissimilarity, 'dissimilarity') + cosineSimilarity(params.homogeneity, 'homogeneity') + cosineSimilarity(params.correlation, 'correlation')) / 6 * 1;
                            total_weight += 1;
                        }
                        if (params.containsKey('hog')) {
                            total_cosineSimilarity += cosineSimilarity(params.hog, 'hog');
                            total_weight += 1;
                        }
                        if (params.containsKey('gist')) {
                            total_cosineSimilarity += cosineSimilarity(params.gist, 'gist');
                            total_weight += 1;
                        }
                        if (params.containsKey('dct')) {
                            total_cosineSimilarity += cosineSimilarity(params.dct, 'dct');
                            total_weight += 1;
                        }
                        if (params.containsKey('wavelet')) {
                            total_cosineSimilarity += cosineSimilarity(params.wavelet, 'wavelet');
                            total_weight += 1;
                        }
                        if (params.containsKey('corners')) {
                            total_cosineSimilarity += cosineSimilarity(params.corners, 'corners');
                            total_weight += 1;
                        }

                        // Compute the average similarity and add 1.0 for Elasticsearch ranking
                        return (total_weight > 0.0) ? (total_cosineSimilarity / total_weight) : 0.0;
                    """,
                    "params": query_features
                }
            }
        }
    }


    # if 'sift' in feature_keys:
    #     sift = compute_sift(image)
    #     query_features['sift'] = [{"vector": vec.tolist()} for vec in sift]
    #         script_score_clauses = []
    # sift = compute_sift(image)

    # # Construct the query for comparing the descriptors
    # script_score_clauses = []
    # for desc in [{"vector": vec.tolist()} for vec in sift]:
    #     print("desc: ", desc)
    #     script_score_clauses.append({
    #         "nested": {
    #             "path": "sift",
    #             "query": {
    #                 "bool": {
    #                     "should": [
    #                         {
    #                             "script_score": {
    #                                 "query": { "match_all": {} },  # Match all documents
    #                                 "script": {
    #                                     "source": """
    #                                         double dist = 0.0;
    #                                         for (int i = 0; i < params.query_vector.length; i++) {
    #                                             dist += Math.pow(params.query_vector[i] - doc['sift.vector'][i], 2);
    #                                         }
    #                                         return Math.sqrt(dist);  // Euclidean distance
    #                                     """,
    #                                     "params": {
    #                                         "query_vector": desc["vector"]
    #                                     }
    #                                 }
    #                             }
    #                         }
    #                     ]
    #                 }
    #             }
    #         }
    #     })

    # # Define the overall query
    # query = {
    #     "size": top_n,
    #     "query": {
    #         "bool": {
    #             "should": script_score_clauses,
    #             "minimum_should_match": 1  # At least one descriptor should match
    #         }
    #     }
    # }

    
    # query = {
    #     "size": top_n,
    #     "query": {
    #         "script_score": {
    #             "query": { "match_all": {} },
    #             "script": {
    #                 "source": """
    #                     double r_mean = params.containsKey('r_mean') ? params.r_mean : 0;
    #                     double g_mean = params.containsKey('g_mean') ? params.g_mean : 0;
    #                     double b_mean = params.containsKey('b_mean') ? params.b_mean : 0;
    #                     double i_mean = params.containsKey('i_mean') ? params.i_mean : 0;

    #                     double doc_r_mean = doc.containsKey('r_mean') ? doc['r_mean'].value : 0;
    #                     double doc_g_mean = doc.containsKey('g_mean') ? doc['g_mean'].value : 0;
    #                     double doc_b_mean = doc.containsKey('b_mean') ? doc['b_mean'].value : 0;
    #                     double doc_i_mean = doc.containsKey('i_mean') ? doc['i_mean'].value : 0;

    #                     // Compute cosine similarity for mean values
    #                     double dot_product = (r_mean * doc_r_mean) + (g_mean * doc_g_mean) + (b_mean * doc_b_mean) + (i_mean * doc_i_mean);
    #                     double doc_magnitude = Math.sqrt(Math.pow(doc_r_mean, 2) + Math.pow(doc_g_mean, 2) + Math.pow(doc_b_mean, 2) + Math.pow(doc_i_mean, 2));
    #                     double query_magnitude = Math.sqrt(Math.pow(r_mean, 2) + Math.pow(g_mean, 2) + Math.pow(b_mean, 2) + Math.pow(i_mean, 2));
    #                     double cosine_similarity = doc_magnitude > 0 && query_magnitude > 0 ? dot_product / (doc_magnitude * query_magnitude) : 0;

    #                     // Compute cosine similarity for histograms
    #                     double histogram_dot_product = 0;
    #                     double histogram_doc_magnitude = 0;
    #                     double histogram_query_magnitude = 0;

    #                     if (params.containsKey('r_hist') && doc.containsKey('r_hist')) {
    #                         for (int i = 0; i < params.r_hist.length; i++) {
    #                             histogram_dot_product += params.r_hist[i] * doc['r_hist'][i];
    #                             histogram_query_magnitude += Math.pow(params.r_hist[i], 2);
    #                             histogram_doc_magnitude += Math.pow(doc['r_hist'][i], 2);
    #                         }
    #                     }
    #                     if (params.containsKey('g_hist') && doc.containsKey('g_hist')) {
    #                         for (int i = 0; i < params.g_hist.length; i++) {
    #                             histogram_dot_product += params.g_hist[i] * doc['g_hist'][i];
    #                             histogram_query_magnitude += Math.pow(params.g_hist[i], 2);
    #                             histogram_doc_magnitude += Math.pow(doc['g_hist'][i], 2);
    #                         }
    #                     }
    #                     if (params.containsKey('b_hist') && doc.containsKey('b_hist')) {
    #                         for (int i = 0; i < params.b_hist.length; i++) {
    #                             histogram_dot_product += params.b_hist[i] * doc['b_hist'][i];
    #                             histogram_query_magnitude += Math.pow(params.b_hist[i], 2);
    #                             histogram_doc_magnitude += Math.pow(doc['b_hist'][i], 2);
    #                         }
    #                     }
    #                     if (params.containsKey('i_hist') && doc.containsKey('i_hist')) {
    #                         for (int i = 0; i < params.i_hist.length; i++) {
    #                             histogram_dot_product += params.i_hist[i] * doc['i_hist'][i];
    #                             histogram_query_magnitude += Math.pow(params.i_hist[i], 2);
    #                             histogram_doc_magnitude += Math.pow(doc['i_hist'][i], 2);
    #                         }
    #                     }

    #                     double histogram_doc_magnitude_sqrt = Math.sqrt(histogram_doc_magnitude);
    #                     double histogram_query_magnitude_sqrt = Math.sqrt(histogram_query_magnitude);
    #                     double histogram_cosine_similarity = (histogram_doc_magnitude_sqrt > 0 && histogram_query_magnitude_sqrt > 0)
    #                         ? histogram_dot_product / (histogram_doc_magnitude_sqrt * histogram_query_magnitude_sqrt)
    #                         : 0;

    #                     // Combine both cosine similarities
    #                     return (cosine_similarity * 0.9999) + (histogram_cosine_similarity * 0.0001);
    #                 """,
    #                 "params": query_features
    #             }
    #         }
    #     }
    # }

    # query = {
    #     "size": top_n,
    #     "query": {
    #         "script_score": {
    #             "query": { "match_all": {} },
    #             "script": {
    #                 "source": """
    #                     double r_mean = params.containsKey('r_mean') ? params.r_mean : 0;
    #                     double g_mean = params.containsKey('g_mean') ? params.g_mean : 0;
    #                     double b_mean = params.containsKey('b_mean') ? params.b_mean : 0;
    #                     double i_mean = params.containsKey('i_mean') ? params.i_mean : 0;

    #                     double doc_r_mean = doc.containsKey('r_mean') ? doc['r_mean'].value : 0;
    #                     double doc_g_mean = doc.containsKey('g_mean') ? doc['g_mean'].value : 0;
    #                     double doc_b_mean = doc.containsKey('b_mean') ? doc['b_mean'].value : 0;
    #                     double doc_i_mean = doc.containsKey('i_mean') ? doc['i_mean'].value : 0;

    #                     // Compute cosine similarity for mean values
    #                     double dot_product = (r_mean * doc_r_mean) + (g_mean * doc_g_mean) + (b_mean * doc_b_mean) + (i_mean * doc_i_mean);
    #                     double doc_magnitude = Math.sqrt(Math.pow(doc_r_mean, 2) + Math.pow(doc_g_mean, 2) + Math.pow(doc_b_mean, 2) + Math.pow(doc_i_mean, 2));
    #                     double query_magnitude = Math.sqrt(Math.pow(r_mean, 2) + Math.pow(g_mean, 2) + Math.pow(b_mean, 2) + Math.pow(i_mean, 2));
    #                     double cosine_similarity = doc_magnitude > 0 && query_magnitude > 0 ? dot_product / (doc_magnitude * query_magnitude) : 0;

    #                     // Compute cosine similarity for histograms
    #                     double histogram_dot_product = 0;
    #                     double histogram_doc_magnitude = 0;
    #                     double histogram_query_magnitude = 0;

    #                     if (params.containsKey('r_hist') && doc.containsKey('r_hist')) {
    #                         for (int i = 0; i < params.r_hist.length; i++) {
    #                             histogram_dot_product += params.r_hist[i] * doc['r_hist'][i];
    #                             histogram_query_magnitude += Math.pow(params.r_hist[i], 2);
    #                             histogram_doc_magnitude += Math.pow(doc['r_hist'][i], 2);
    #                         }
    #                     }
    #                     if (params.containsKey('g_hist') && doc.containsKey('g_hist')) {
    #                         for (int i = 0; i < params.g_hist.length; i++) {
    #                             histogram_dot_product += params.g_hist[i] * doc['g_hist'][i];
    #                             histogram_query_magnitude += Math.pow(params.g_hist[i], 2);
    #                             histogram_doc_magnitude += Math.pow(doc['g_hist'][i], 2);
    #                         }
    #                     }
    #                     if (params.containsKey('b_hist') && doc.containsKey('b_hist')) {
    #                         for (int i = 0; i < params.b_hist.length; i++) {
    #                             histogram_dot_product += params.b_hist[i] * doc['b_hist'][i];
    #                             histogram_query_magnitude += Math.pow(params.b_hist[i], 2);
    #                             histogram_doc_magnitude += Math.pow(doc['b_hist'][i], 2);
    #                         }
    #                     }
    #                     if (params.containsKey('i_hist') && doc.containsKey('i_hist')) {
    #                         for (int i = 0; i < params.i_hist.length; i++) {
    #                             histogram_dot_product += params.i_hist[i] * doc['i_hist'][i];
    #                             histogram_query_magnitude += Math.pow(params.i_hist[i], 2);
    #                             histogram_doc_magnitude += Math.pow(doc['i_hist'][i], 2);
    #                         }
    #                     }

    #                     double histogram_doc_magnitude_sqrt = Math.sqrt(histogram_doc_magnitude);
    #                     double histogram_query_magnitude_sqrt = Math.sqrt(histogram_query_magnitude);
    #                     double histogram_cosine_similarity = (histogram_doc_magnitude_sqrt > 0 && histogram_query_magnitude_sqrt > 0)
    #                         ? histogram_dot_product / (histogram_doc_magnitude_sqrt * histogram_query_magnitude_sqrt)
    #                         : 0;

    #                     // Compute cosine similarity for GLCM features
    #                     String[] glcm_features = {"energy", "contrast", "entropy", "dissimilarity", "homogeneity", "correlation"};
    #                     double glcm_similarity_sum = 0;
    #                     int glcm_feature_count = 0;

    #                     for (String feature : glcm_features) {
    #                         if (params.containsKey(feature) && doc.containsKey(feature)) {
    #                             double glcm_dot_product = 0;
    #                             double glcm_query_magnitude = 0;
    #                             double glcm_doc_magnitude = 0;

    #                             for (int i = 0; i < params[feature].length; i++) {
    #                                 glcm_dot_product += params[feature][i] * doc[feature][i];
    #                                 glcm_query_magnitude += Math.pow(params[feature][i], 2);
    #                                 glcm_doc_magnitude += Math.pow(doc[feature][i], 2);
    #                             }

    #                             double glcm_doc_magnitude_sqrt = Math.sqrt(glcm_doc_magnitude);
    #                             double glcm_query_magnitude_sqrt = Math.sqrt(glcm_query_magnitude);
    #                             double glcm_cosine_similarity = (glcm_doc_magnitude_sqrt > 0 && glcm_query_magnitude_sqrt > 0)
    #                                 ? glcm_dot_product / (glcm_doc_magnitude_sqrt * glcm_query_magnitude_sqrt)
    #                                 : 0;

    #                             glcm_similarity_sum += glcm_cosine_similarity;
    #                             glcm_feature_count++;
    #                         }
    #                     }

    #                     double glcm_similarity_avg = glcm_feature_count > 0 ? glcm_similarity_sum / glcm_feature_count : 0;

    #                     // Combine both cosine similarities
    #                     return (cosine_similarity * 0.8999) + (histogram_cosine_similarity * 0.0001) + (glcm_similarity_avg * 0.10);
    #                 """,
    #                 "params": query_features
    #             }
    #         }
    #     }
    # }
    # query = {
    #     "size": top_n,
    #     "query": {
    #         "script_score": {
    #             "query": {"match_all": {}},
    #             "script": {
    #                 "source": """
    #                     double r_mean = params.containsKey('r_mean') ? params.r_mean : 0;
    #                     double g_mean = params.containsKey('g_mean') ? params.g_mean : 0;
    #                     double b_mean = params.containsKey('b_mean') ? params.b_mean : 0;
    #                     double i_mean = params.containsKey('i_mean') ? params.i_mean : 0;

    #                     double doc_r_mean = doc.containsKey('r_mean') ? doc['r_mean'].value : 0;
    #                     double doc_g_mean = doc.containsKey('g_mean') ? doc['g_mean'].value : 0;
    #                     double doc_b_mean = doc.containsKey('b_mean') ? doc['b_mean'].value : 0;
    #                     double doc_i_mean = doc.containsKey('i_mean') ? doc['i_mean'].value : 0;

    #                     // Compute cosine similarity for mean values
    #                     double dot_product = (r_mean * doc_r_mean) + (g_mean * doc_g_mean) + (b_mean * doc_b_mean) + (i_mean * doc_i_mean);
    #                     double doc_magnitude = Math.sqrt(Math.pow(doc_r_mean, 2) + Math.pow(doc_g_mean, 2) + Math.pow(doc_b_mean, 2) + Math.pow(doc_i_mean, 2));
    #                     double query_magnitude = Math.sqrt(Math.pow(r_mean, 2) + Math.pow(g_mean, 2) + Math.pow(b_mean, 2) + Math.pow(i_mean, 2));
    #                     double cosine_similarity = (doc_magnitude > 0 && query_magnitude > 0) ? dot_product / (doc_magnitude * query_magnitude) : 0;

    #                     // Compute cosine similarity for histograms
    #                     double histogram_dot_product = 0;
    #                     double histogram_doc_magnitude = 0;
    #                     double histogram_query_magnitude = 0;

    #                     if (params.containsKey('r_hist') && doc.containsKey('r_hist')) {
    #                         for (int i = 0; i < params.r_hist.length; i++) {
    #                             histogram_dot_product += params.r_hist[i] * doc['r_hist'][i];
    #                             histogram_query_magnitude += Math.pow(params.r_hist[i], 2);
    #                             histogram_doc_magnitude += Math.pow(doc['r_hist'][i], 2);
    #                         }
    #                     }

    #                     double histogram_doc_magnitude_sqrt = Math.sqrt(histogram_doc_magnitude);
    #                     double histogram_query_magnitude_sqrt = Math.sqrt(histogram_query_magnitude);
    #                     double histogram_cosine_similarity = (histogram_doc_magnitude_sqrt > 0 && histogram_query_magnitude_sqrt > 0) 
    #                         ? histogram_dot_product / (histogram_doc_magnitude_sqrt * histogram_query_magnitude_sqrt) 
    #                         : 0;

    #                     // Compute cosine similarity for GLCM features
    #                     String[] glcm_features = new String[]{"energy", "contrast", "entropy", "dissimilarity", "homogeneity", "correlation"};
    #                     double glcm_similarity_sum = 0;
    #                     int glcm_feature_count = 0;

    #                     for (String feature : glcm_features) {
    #                         if (params.containsKey(feature) && doc.containsKey(feature)) {
    #                             double glcm_dot_product = 0;
    #                             double glcm_query_magnitude = 0;
    #                             double glcm_doc_magnitude = 0;

    #                             for (int i = 0; i < params[feature].length; i++) {
    #                                 glcm_dot_product += params[feature][i] * doc[feature][i];
    #                                 glcm_query_magnitude += Math.pow(params[feature][i], 2);
    #                                 glcm_doc_magnitude += Math.pow(doc[feature][i], 2);
    #                             }

    #                             double glcm_doc_magnitude_sqrt = Math.sqrt(glcm_doc_magnitude);
    #                             double glcm_query_magnitude_sqrt = Math.sqrt(glcm_query_magnitude);
    #                             double glcm_cosine_similarity = (glcm_doc_magnitude_sqrt > 0 && glcm_query_magnitude_sqrt > 0)
    #                                 ? glcm_dot_product / (glcm_doc_magnitude_sqrt * glcm_query_magnitude_sqrt)
    #                                 : 0;

    #                             glcm_similarity_sum += glcm_cosine_similarity;
    #                             glcm_feature_count++;
    #                         }
    #                     }

    #                     double glcm_similarity_avg = glcm_feature_count > 0 ? glcm_similarity_sum / glcm_feature_count : 0;

    #                     // Combine all cosine similarities
    #                     return (cosine_similarity * 0.79999) + (histogram_cosine_similarity * 0.00001) + (glcm_similarity_avg * 0.2);
    #                 """,
    #                 "params": query_features
    #             }
    #         }
    #     }
    # }
    # queyr = {
    # "size": 10,
    # "query": {
    #     "script_score": {
    #     "query": { "match_all": {} },
    #     "script": {
    #         "source": "cosineSimilarity(params.query_vector, 'hog') + 1.0",
    #         "params": {
    #         "query_vector": [ 0.290379, 0.080451, 0.013969, 0.011463, 0.010763, 0.268159, 0.200296, 0.290379, 0.290379, 0.290379, 0.068664, 0.085273, 0.290379, 0.290379, 0.008311, 0.014914, 0.044402, 0.290379, 0.290379, 0.290379, 0.139922, 0.094543, 0.055394, 0.276305, 0.215007, 0.273597, 0.273597, 0.273597, 0.073707, 0.091535, 0.234959, 0.224219, 0.273597, 0.273597, 0.156795, 0.160677, 0.273597, 0.273597, 0.150199, 0.101487, 0.059462, 0.273597, 0.273597, 0.036824, 0.030814, 0.080832, 0.050272, 0.250241, 0.297774, 0.284162, 0.340105, 0.340105, 0.198713, 0.203633, 0.025065, 0.180411, 0.340105, 0.340105, 0.015103, 0.051996, 0.340105, 0.046669, 0.039052, 0.102442, 0.063712, 0.317141, 0.008934, 0.009432, 0.010323, 0.125363, 0.017661, 0.005196, 0.04096, 0.294815, 0.4713, 0.4713, 0.02468, 0.084969, 0.078339, 0.011957, 0.411338, 0.4713, 0.038314, 0.01934, 0.0146, 0.015414, 0.016869, 0.204859, 0.02886, 0.008492, 0.029777, 0.003836, 0.002776, 0.096619, 0.064436, 0.043392, 0.111386, 0.017001, 0.432711, 0.432711, 0.054476, 0.027498, 0.174277, 0.070809, 0.432711, 0.432711, 0.377007, 0.118812, 0.042338, 0.005454, 0.003947, 0.137377, 0.091618, 0.061696, 0.052569, 0.018448, 0.023409, 0.057178, 0.048948, 0.027589, 0.133332, 0.054173, 0.30759, 0.30759, 0.288433, 0.090898, 0.30759, 0.140503, 0.245608, 0.30759, 0.30759, 0.30759, 0.040218, 0.014113, 0.017909, 0.043745, 0.037448, 0.021107, 0.232631, 0.090207, 0.008141, 0.146227, 0.2364, 0.30759, 0.284211, 0.164586, 0.284211, 0.284211, 0.284211, 0.284211, 0.284211, 0.199413, 0.002645, 0.027907, 0.091818, 0.26038, 0.272504, 0.105668, 0.009536, 0.171291, 0.276918, 0.284211, 0.248395, 0.013329, 0.026266, 0.053563, 0.143728, 0.114307, 0.276146, 0.276146, 0.009064, 0.016266, 0.048425, 0.276146, 0.276146, 0.276146, 0.152599, 0.103109, 0.060412, 0.276146, 0.261999, 0.18887, 0.143012, 0.180604, 0.18029, 0.190123, 0.276146, 0.181415, 0.137292, 0.223141, 0.195291, 0.245389, 0.278353, 0.278353, 0.190581, 0.128772, 0.075449, 0.278353, 0.278353, 0.046725, 0.039098, 0.102564, 0.063788, 0.278353, 0.278353, 0.226569, 0.171464, 0.278353, 0.243898, 0.278353, 0.278353, 0.022887, 0.074135, 0.041411, 0.018624, 0.278353, 0.387547, 0.10932, 0.091477, 0.239966, 0.149243, 0.387547, 0.020928, 0.022095, 0.024182, 0.293657, 0.041369, 0.012173, 0.387547, 0.053547, 0.173452, 0.096888, 0.043574, 0.387547, 0.013089, 0.055911, 0.361615, 0.168501, 0.052099, 0.00622, 0.059571, 0.062892, 0.068832, 0.364805, 0.117756, 0.034649, 0.121499, 0.015652, 0.011328, 0.364805, 0.262916, 0.177051, 0.037256, 0.159149, 0.364805, 0.364805, 0.148298, 0.017705, 0.05483, 0.050209, 0.275965, 0.364805, 0.239024, 0.061898, 0.07082, 0.009123, 0.006603, 0.229791, 0.153249, 0.1032, 0.087932, 0.030857, 0.039156, 0.095642, 0.081876, 0.046149, 0.03196, 0.029266, 0.160856, 0.401788, 0.139324, 0.03608, 0.214578, 0.401788, 0.401788, 0.401788, 0.324758, 0.184411, 0.047848, 0.016791, 0.021306, 0.052043, 0.044552, 0.025111, 0.27676, 0.107319, 0.009685, 0.173966, 0.281244, 0.317508, 0.11676, 0.316749, 0.317508, 0.317508, 0.176714, 0.100346, 0.092949, 0.053563, 0.317508, 0.317508, 0.316526, 0.151878, 0.292585, 0.113455, 0.010239, 0.183913, 0.297325, 0.297777, 0.266699, 0.014311, 0.028202, 0.05751, 0.15432, 0.122731, 0.098263, 0.056625, 0.297777, 0.297777, 0.297777, 0.160562, 0.277538, 0.039487, 0.049777, 0.114649, 0.297777, 0.297777, 0.236104, 0.234253, 0.177376, 0.224, 0.223612, 0.235806, 0.236104, 0.225007, 0.170282, 0.236104, 0.236104, 0.236104, 0.140647, 0.136845, 0.117075, 0.236104, 0.236104, 0.236104, 0.176541, 0.049444, 0.072738, 0.234898, 0.236104, 0.17394, 0.265539, 0.24866, 0.188182, 0.265539, 0.265539, 0.265539, 0.265539, 0.025118, 0.081364, 0.045449, 0.02044, 0.265539, 0.195099, 0.054642, 0.080384, 0.259591, 0.265539, 0.192225, 0.265539, 0.072862, 0.265539, 0.173401, 0.026231, 0.265539, 0.339096, 0.038087, 0.123373, 0.068914, 0.030994, 0.339096, 0.00931, 0.039769, 0.257209, 0.119851, 0.037057, 0.004424, 0.339096, 0.110482, 0.339096, 0.26293, 0.039775, 0.339096, 0.023871, 0.023739, 0.339096, 0.339096, 0.053305, 0.047307, 0.012093, 0.051658, 0.334109, 0.155684, 0.048136, 0.005747, 0.017797, 0.016297, 0.089576, 0.29905, 0.077585, 0.020092, 0.031008, 0.030836, 0.370724, 0.370724, 0.069241, 0.061451, 0.157602, 0.370724, 0.370724, 0.370724, 0.081062, 0.15948, 0.013266, 0.012148, 0.066768, 0.222905, 0.05783, 0.014976, 0.089066, 0.24162, 0.317153, 0.317153, 0.1348, 0.076545, 0.117473, 0.317153, 0.317153, 0.317153, 0.060422, 0.118872, 0.088406, 0.317153, 0.317153, 0.317153, 0.068551, 0.053233, 0.091245, 0.24753, 0.310735, 0.310735, 0.138097, 0.078417, 0.072637, 0.041858, 0.310735, 0.310735, 0.247356, 0.118689, 0.090568, 0.310735, 0.310735, 0.310735, 0.070228, 0.054535, 0.059785, 0.031021, 0.310735, 0.169206, 0.031506, 0.012828, 0.089434, 0.051537, 0.304479, 0.304479, 0.304479, 0.146134, 0.252599, 0.035939, 0.045304, 0.104347, 0.304479, 0.304479, 0.07361, 0.038194, 0.304479, 0.208334, 0.038792, 0.015794, 0.098305, 0.01875, 0.304479, 0.206261, 0.209043, 0.304479, 0.162805, 0.158405, 0.13552, 0.26746, 0.26746, 0.26746, 0.204355, 0.057234, 0.084198, 0.26746, 0.26746, 0.201344, 0.163037, 0.051476, 0.049442, 0.175925, 0.26746, 0.26746, 0.161774, 0.149295, 0.104589, 0.26746, 0.26746, 0.26746, 0.203247, 0.056924, 0.083741, 0.270432, 0.274782, 0.200252, 0.274782, 0.075905, 0.274782, 0.180642, 0.027327, 0.274782, 0.160897, 0.148486, 0.104022, 0.274782, 0.274782, 0.274782, 0.102197, 0.08659, 0.274086, 0.274782, 0.151342, 0.131504, 0.276981, 0.088006, 0.276981, 0.20944, 0.031683, 0.276981, 0.019015, 0.018909, 0.276981, 0.276981, 0.04246, 0.037683, 0.118489, 0.100395, 0.276981, 0.276981, 0.175469, 0.152468, 0.233737, 0.14663, 0.258729, 0.276981, 0.23544, 0.230297, 0.013625, 0.013549, 0.207143, 0.296422, 0.030424, 0.027001, 0.069249, 0.285362, 0.296422, 0.224193, 0.035618, 0.070073, 0.167477, 0.105063, 0.185385, 0.207431, 0.168698, 0.165013, 0.276145, 0.296422, 0.296422, 0.296422, 0.22539, 0.27174, 0.051055, 0.21039, 0.280134, 0.165292, 0.02626, 0.051663, 0.038422, 0.209939, 0.164577, 0.25062, 0.029793, 0.023136, 0.203595, 0.280134, 0.280134, 0.280134, 0.166175, 0.200347, 0.246165, 0.140426, 0.280134, 0.280134, 0.267786, 0.231301, 0.047317, 0.258539, 0.202676, 0.277144, 0.03669, 0.028492, 0.031235, 0.016207, 0.277144, 0.088401, 0.01646, 0.006702, 0.277144, 0.172934, 0.277144, 0.277144, 0.277144, 0.277144, 0.223047, 0.190148, 0.277144, 0.227409, 0.217567, 0.221313, 0.048789, 0.025315, 0.275668, 0.138085, 0.025712, 0.010468, 0.065157, 0.012428, 0.221176, 0.13671, 0.138555, 0.275668, 0.275668, 0.275668, 0.275668, 0.275668, 0.275668, 0.275668, 0.149225, 0.207214, 0.275668, 0.275668, 0.156043, 0.190929, 0.178504, 0.05636, 0.054132, 0.192615, 0.290783, 0.290783, 0.177122, 0.163458, 0.114511, 0.290783, 0.290783, 0.290783, 0.057514, 0.013375, 0.01714, 0.067865, 0.257534, 0.163388, 0.290783, 0.133035, 0.097556, 0.220603, 0.290783, 0.290783, 0.142697, 0.131689, 0.092255, 0.273349, 0.291638, 0.267368, 0.090637, 0.076795, 0.243082, 0.291638, 0.134223, 0.116628, 0.291638, 0.107179, 0.078595, 0.177728, 0.291638, 0.291638, 0.102205, 0.083891, 0.291638, 0.291638, 0.135567, 0.149251, 0.105633, 0.089502, 0.283302, 0.304256, 0.156431, 0.135926, 0.208376, 0.13072, 0.230657, 0.258086, 0.209895, 0.20531, 0.119115, 0.097771, 0.304256, 0.304256, 0.157998, 0.173946, 0.07001, 0.01243, 0.304256, 0.304256, 0.177948, 0.112018, 0.150182, 0.094214, 0.166241, 0.18601, 0.151277, 0.147972, 0.247628, 0.309767, 0.309767, 0.309767, 0.202115, 0.243678, 0.050458, 0.008959, 0.309767, 0.309767, 0.128252, 0.080734, 0.010034, 0.003916, 0.309767, 0.309767, 0.051027, 0.0, 0.185507, 0.285466, 0.286271, 0.286271, 0.151411, 0.182548, 0.224296, 0.12795, 0.286271, 0.286271, 0.243995, 0.210752, 0.007517, 0.002934, 0.286271, 0.286271, 0.038226, 0.0, 0.004946, 0.003348, 0.286271, 0.286271, 0.006846, 0.001309, 0.277593, 0.158354, 0.284104, 0.284104, 0.284104, 0.260831, 0.204241, 0.174117, 0.27572, 0.208236, 0.199224, 0.202654, 0.006121, 0.004143, 0.284104, 0.284104, 0.008472, 0.00162, 0.00502, 0.018535, 0.284104, 0.268999, 0.021851, 0.005832, 0.266878, 0.230289, 0.266878, 0.266878, 0.263496, 0.266878, 0.115701, 0.160662, 0.266878, 0.266878, 0.120987, 0.148036, 0.006639, 0.024514, 0.266878, 0.266878, 0.028901, 0.007714, 0.266878, 0.183542, 0.266878, 0.151557, 0.086375, 0.154553, 0.062036, 0.014427, 0.018488, 0.073201, 0.277784, 0.176234, 0.304819, 0.143495, 0.105226, 0.237949, 0.304819, 0.304819, 0.024067, 0.00728, 0.05423, 0.223095, 0.304819, 0.290766, 0.304819, 0.117424, 0.027011, 0.085676, 0.276591, 0.304819, 0.321312, 0.128403, 0.094159, 0.212922, 0.321312, 0.321312, 0.122444, 0.100503, 0.321312, 0.321312, 0.162413, 0.178807, 0.321312, 0.105074, 0.02417, 0.076665, 0.247501, 0.321312, 0.066196, 0.009893, 0.029204, 0.138319, 0.11038, 0.088137, 0.17296, 0.141968, 0.390268, 0.390268, 0.22942, 0.252577, 0.101657, 0.01805, 0.390268, 0.390268, 0.258389, 0.162655, 0.093507, 0.013975, 0.041253, 0.195386, 0.155919, 0.1245, 0.003563, 0.010076, 0.103732, 0.144445, 0.004211, 0.0, 0.104964, 0.018637, 0.445587, 0.445587, 0.266793, 0.167946, 0.020874, 0.008146, 0.445587, 0.445587, 0.106148, 0.0, 0.003679, 0.010403, 0.107106, 0.149143, 0.004348, 0.0, 0.002647, 0.004348, 0.093393, 0.201691, 0.002767, 0.0, 0.020702, 0.008079, 0.468371, 0.468371, 0.105275, 0.0, 0.01362, 0.009219, 0.468371, 0.468371, 0.018853, 0.003605, 0.002625, 0.004312, 0.092625, 0.200032, 0.002744, 0.0, 0.002283, 0.020662, 0.205825, 0.133137, 0.01451, 0.030654, 0.014939, 0.010112, 0.413738, 0.413738, 0.020678, 0.003954, 0.012251, 0.045236, 0.413738, 0.413738, 0.053331, 0.014235, 0.002504, 0.022662, 0.225751, 0.146026, 0.015914, 0.033621, 0.195957, 0.11463, 0.309082, 0.17269, 0.005819, 0.24115, 0.009416, 0.034769, 0.318189, 0.318189, 0.040991, 0.010941, 0.318189, 0.260323, 0.318189, 0.214959, 0.122509, 0.219208, 0.150615, 0.088106, 0.237564, 0.132731, 0.004473, 0.18535, 0.318189, 0.318189, 0.152183, 0.087261, 0.031732, 0.204312, 0.025284, 0.007648, 0.056974, 0.234383, 0.312106, 0.305478, 0.312106, 0.123365, 0.028378, 0.090011, 0.290586, 0.312106, 0.051631, 0.016755, 0.010162, 0.031939, 0.312106, 0.175291, 0.312106, 0.095485, 0.102414, 0.054457, 0.312106, 0.312106, 0.359144, 0.168833, 0.038837, 0.123186, 0.359144, 0.359144, 0.106364, 0.015896, 0.046925, 0.222252, 0.177359, 0.14162, 0.359144, 0.130677, 0.140161, 0.074528, 0.359144, 0.359144, 0.073277, 0.001742, 0.049505, 0.12088, 0.011614, 0.037203, 0.236478, 0.035342, 0.104328, 0.280442, 0.280442, 0.280442, 0.00901, 0.025481, 0.262338, 0.280442, 0.010649, 0.0, 0.162917, 0.003872, 0.110063, 0.268752, 0.025821, 0.082713, 0.280442, 0.227905, 0.250498, 0.280442, 0.280442, 0.280442, 0.01043, 0.029498, 0.303686, 0.31214, 0.012327, 0.0, 0.007506, 0.012327, 0.264804, 0.31214, 0.007845, 0.0, 0.31214, 0.263826, 0.28998, 0.31214, 0.31214, 0.31214, 0.005547, 0.002241, 0.0, 0.31214, 0.022493, 0.0, 0.010715, 0.017598, 0.353267, 0.353267, 0.011198, 0.0, 0.009317, 0.084326, 0.353267, 0.353267, 0.059216, 0.125103, 0.007919, 0.0032, 0.0, 0.353267, 0.032109, 0.0, 0.226916, 0.109196, 0.192685, 0.353267, 0.344936, 0.060696, 0.00506, 0.045798, 0.286361, 0.286361, 0.03216, 0.067944, 0.286361, 0.231652, 0.286361, 0.286361, 0.01176, 0.286361, 0.123239, 0.059305, 0.104648, 0.286361, 0.187336, 0.032964, 0.286361, 0.083665, 0.120438, 0.212701, 0.286361, 0.260417, 0.173349, 0.101405, 0.273423, 0.152767, 0.005148, 0.213328, 0.310373, 0.310373, 0.175154, 0.100433, 0.036522, 0.235152, 0.210905, 0.036624, 0.052722, 0.093109, 0.212388, 0.113997, 0.310373, 0.310373, 0.105146, 0.119239, 0.310373, 0.310373]
    #         }
    #     }
    #     }
    # }
    # }
    
    print("query_features: ", query_features)
    print("Query: ", query)

    response = es.search(index=ELASTIC_INDEX, body=query)
    similar_images = [{
        'file': hit["_source"]["file"],
        '_score': hit["_score"],
    } for hit in response["hits"]["hits"]]

    print("Took: ", response['took'])
    
    print("Top similar images:")
    plt.figure(figsize=(10, 20))
    plt.subplot(6, 2, 1)
    plt.imshow(image_rgb)
    plt.title("Original Image")
    plt.axis("off")
    
    i = 3
    for img in similar_images:
        # print(img)
        image = f'dataset/{img["file"]}'
        image_temp = cv2.imread(image)
        image_temp = cv2.cvtColor(image_temp, cv2.COLOR_BGR2RGB)
        plt.subplot(6, 2, i)
        plt.imshow(image_temp)
        plt.title(f"Item: {i-2} - Score {img['_score']}")
        plt.axis("off")
        i += 1

    plt.tight_layout()
    plt.show()
    
    return similar_images

search_similar_images('dataset/windows-184_8465.JPG')


In [ ]:
# Q V.1
    # query = {
    #     "size": top_n,
    #     "query": {
    #         "script_score": {
    #             "query": {"match_all": {}},
    #             "script": {
    #                 "source": """
    #                     // Vector from the request
    #                     double r_mean = params.r_mean;
    #                     double g_mean = params.g_mean;
    #                     double b_mean = params.b_mean;
    #                     double i_mean = params.i_mean;

    #                     // Vector from the document
    #                     double doc_r_mean = doc['r_mean'].value;
    #                     double doc_g_mean = doc['g_mean'].value;
    #                     double doc_b_mean = doc['b_mean'].value;
    #                     double doc_i_mean = doc['i_mean'].value;

    #                     // Compute the dot product
    #                     double dot_product = (r_mean * doc_r_mean) + (g_mean * doc_g_mean) + (b_mean * doc_b_mean) + (i_mean * doc_i_mean);

    #                     // Compute the magnitude of the vectors
    #                     double doc_magnitude = Math.sqrt(Math.pow(doc_r_mean, 2) + Math.pow(doc_g_mean, 2) + Math.pow(doc_b_mean, 2) + Math.pow(doc_i_mean, 2));
    #                     double query_magnitude = Math.sqrt(Math.pow(r_mean, 2) + Math.pow(g_mean, 2) + Math.pow(b_mean, 2) + Math.pow(i_mean, 2));

    #                     // Compute the cosine similarity (dot product / (magnitude of query * magnitude of document))
    #                     double cosine_similarity = dot_product / (doc_magnitude * query_magnitude);

    #                     return cosine_similarity;
    #                 """,
    #                 "params": query_features
    #             }
    #         }
    #     }
    # }